In [1]:
import pandas as pd
import numpy as np

In [2]:
OHCO = ['doc_source','doc_id', 'sent_num', 'token_num']
articles = OHCO[:2]
sentences = OHCO[:3]

In [3]:
TOKEN = pd.read_csv("TOKEN.csv").set_index(OHCO)
VOCAB = pd.read_csv("VOCAB.csv").set_index('term_id')
TOKEN.head()

token  pos term_str  term_id
doc_source doc_id sent_num token_num                                
PowerLine  48     1        1             I’ve  VBN      ive    85795
                           2             been  VBG     been    17768
                           3          falling   IN  falling    61167
                           4           behind   IN   behind    18021
                           5               on   JJ       on   114185

In [4]:
VOCAB = VOCAB[~VOCAB.term_str.isna()]
TOKEN = TOKEN[~TOKEN.term_str.isna()]

Note: For the sake of computer limitations, I'm deleting infrequently used words and stop words from the token table in order to compute tf-idf.

In [5]:
VOCAB.n.describe()

count    1.799370e+05
mean     1.088474e+02
std      3.984836e+03
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      6.000000e+00
max      1.103705e+06
Name: n, dtype: float64

In [6]:
VSHORT = VOCAB[VOCAB.n > 5]

In [7]:
len(VSHORT)


47636

In [8]:
VSHORT

,term_rank,term_str,n,stop,p_stem,pos_max
term_id,,,,,,
158064,1,the,1103705,1,the,NN
160412,2,to,558006,1,to,VB
113256,3,of,509160,1,of,DT
10479,4,and,451366,1,and,NNP
4833,5,a,439586,1,a,NN
...,...,...,...,...,...,...
138179,47635,rudolf,6,0,rudolf,NNP
92706,47636,laurent,6,0,laurent,NNP
126176,47637,princely,6,0,princ,NN


In [9]:
len(TOKEN)

19585678

In [10]:
TOKEN = TOKEN[TOKEN.term_id.isin(VSHORT.index)]

In [11]:
len(TOKEN)

19381620

In [12]:
VSHORT = VSHORT[VSHORT.stop != 1]
TOKEN2 = TOKEN[TOKEN.term_id.isin(VSHORT.index)]

In [13]:
len(TOKEN2)

10923588

In [14]:
def createTFIDF(TOKEN, bag=articles, count_method='c', tf_method='raw', idf_method='standard'):
    '''Creates TFIDF table using specified method'''
    BOW = TOKEN.groupby(bag+['term_id']).term_id.count()\
    .to_frame().rename(columns={'term_id':'n'})
    BOW['c'] = BOW.n.astype('bool').astype('int')
    DTCM = BOW[count_method].unstack().fillna(0).astype('int')
    
    print('TF method:', tf_method)
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = tf_norm_k + (1 - tf_norm_k) * TF[TF > 0] 
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
        
    TF = TF.T
    DF = DTCM[DTCM > 0].count()
    N = DTCM.shape[0]
    
    print('IDF method:', idf_method)
    if idf_method == 'standard':
        IDF = np.log10(N / DF)
    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 
    elif idf_method == 'smooth':
        IDF = np.log10(N / (1 + DF)) + 1 
    elif idf_method == 'prob':  #Probabilistic method as described by wikipedia
        IDF = np.log10((N - DF) / DF)
    
    TFIDF = TF * IDF
    return TFIDF

#### Generate a few TFIDF tables using different methods to pick one

In [15]:
TFIDF1 = createTFIDF(TOKEN2)

TF method: raw
IDF method: standard


In [16]:
#TFIDF2 = createTFIDF(TOKEN2, tf_method='sum', idf_method='prob')

In [22]:
VOCAB['tfidf_sum'] = TFIDF1.sum().copy()
VSHORT['tfidf_sum'] = TFIDF1.sum().copy()

/Users/matthewthomas/anaconda3/envs/DS5001/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
VOCAB.sort_values('tfidf_sum', ascending=False).head(20).style.background_gradient(cmap='PuBuGn')

,term_rank,term_str,n,stop,p_stem,pos_max,tfidf_sum
term_id,,,,,,,
100438,126,may,17445,0,may,VB,3973.093033
90443,123,know,17578,0,know,IN,3972.717577
15427,127,back,17300,0,back,IN,3971.772548
158960,112,think,18632,0,think,IN,3970.629391
136284,117,right,18361,0,right,NN,3969.627104
106902,113,much,18619,0,much,IN,3969.558610
92444,110,last,18991,0,last,NN,3967.986151
78366,74,house,28349,0,hous,NNP,3967.920868
171898,118,way,18147,0,way,IN,3967.722582


In [22]:
VOCAB[['term_rank','term_str','pos_max','tfidf_sum']]\
    .sort_values('tfidf_sum', ascending=False).head(50)\
    .style.background_gradient(cmap='PuBuGn', high=1)

,term_rank,term_str,pos_max,tfidf_sum
term_id,,,,
100438,126,may,VB,3973.093033
90443,123,know,IN,3972.717577
15427,127,back,IN,3971.772548
158960,112,think,IN,3970.629391
136284,117,right,NN,3969.627104
106902,113,much,IN,3969.558610
92444,110,last,NN,3967.986151
78366,74,house,NNP,3967.920868
171898,118,way,IN,3967.722582


## Visualize

In [17]:
import plotly_express as px

In [24]:
px.scatter(VSHORT, x='term_rank', y='tfidf_sum', hover_name='term_str', color='pos_max')

KeyError: (nan, '', '', '', '')

In [27]:
BOW = TOKEN2.groupby(articles+['term_id']).term_id.count()\
    .to_frame().rename(columns={'term_id':'n'})

In [ ]:
BOW['c'] = BOW.n.astype('bool').astype('int')
BOW['tfidf'] = TFIDF1.stack()

In [31]:
#create doc-term matrix
DCTM = BOW['c'].unstack().fillna(0).astype('int')

In [ ]:
WCM = DTCM / DTCM.sum()

WCM.sum().head()

WCMh = WCM * np.log2(1/WCM)

VOCAB['h2'] = WCMh.sum()

VOCAB['h2'].hist()

In [38]:
VSHORT.to_csv('VOCAB2.csv')

In [39]:
TOKEN2.to_csv('TOKEN2.csv')

In [16]:
TFIDF1.to_csv('TFIDF.csv')

SystemError: <built-in method item of numpy.ndarray object at 0x1e247c850> returned a result with an error set